In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import json
import re
import nibabel 
from nilearn.glm.first_level import run_glm
from nilearn.glm.contrasts import compute_contrast
from nilearn.glm.thresholding import fdr_threshold
from nilearn import image as nimg
from nilearn import plotting 
from nilearn.plotting import view_surf
from nilearn.plotting import plot_surf_stat_map
%matplotlib inline
import hcp_utils as hcp
from hcp_utils import left_cortex_data, right_cortex_data
import matplotlib.pyplot as plt

In [ ]:
#get 3 group means

In [ ]:
def get_group_average_contrast(contrast,ses,group,task,covariates,subgroup,rm_CUD,desired_outliers):
    print('Now processing: '+ ' '.join([contrast,ses,group,task]))
    
    #check if mriqc summary exists and find runs that need to be excluded based on desired outlier cutoffs
    if desired_outliers:
        if len(glob.glob(f'../../../derivatives/mriqc_summaries/group-{group}_ses-{ses}_task-{task}_rec-unco.tsv'))==0:
            print('No mriqc summary found. Please create this before requesting to remove outliers.')
            return (None, None)

    excluded_subs_runs_dict = get_excluded_runs(desired_outliers,contrast,ses,group,task)

    effect_size_maps, all_subs = get_effect_size_maps(contrast,ses,group,task,excluded_subs_runs_dict)
    
    
    #remove participants with CUD at baseline (only applies to MM)
    if rm_CUD:
        effect_size_maps, all_subs = rm_CUD_baseline(effect_size_maps, all_subs)

        
    #ensure we have data
    if not effect_size_maps:
        print('No effect size maps, so cannot generate group level output for: '+' '.join([contrast,ses,group,task]))
        return (None, None)
    
    #note that subject formatting is changed slightly to match that of the non_imaging_data csv file
    #need these to make design matrix
    subs_csv = ['_'.join([s for s in re.split(r'(MM|HC)', sub) if s]) for sub in all_subs]
    
    
    if subgroup:
        effect_size_maps, subs_csv, all_subs = select_subgroup(subgroup, effect_size_maps, subs_csv, all_subs)

    
    #needed for printing total later
    part_count = len(subs_csv)
    
    #create design matrix including all covariates and a way to encode group average (either as col of 1s or with Male/Female)
    group_design_matrix = create_group_design_matrix(subs_csv,group,ses,covariates)
    display(group_design_matrix)
    

    effect_size_signals = [nimg.load_img(dscalar).get_fdata(dtype='f4') for dscalar in effect_size_maps]
        
    effect_size_signals_combined = np.vstack(effect_size_signals)
    
    


    #set up and run glm
    #note that we are not smoothing with nilearn since the data was smoothed using the HCP pipeline
    labels, estimates = run_glm(effect_size_signals_combined, group_design_matrix.values, noise_model='ols', 
                                n_jobs=-2, verbose=0)

    
    #define contrast matrix 
    contrast_matrix = np.eye(group_design_matrix.shape[1])
    contrasts = dict([(column, contrast_matrix[i])
                      for i, column in enumerate(group_design_matrix.columns)])

    #here we focus on group average or sex if included instead of group average
    #can instead insert various slopes here if desired to plot their slopes instead
    if 'sex' in covariates:
        grp_contrast = 0.5*contrasts['Female']+0.5*contrasts['Male']
    else:
        grp_contrast = contrasts['group_average']
        
    print(grp_contrast)
    
    
    #compute the group average contrast
    #pick stat_type none because it automatically selects t vs. F test 
    contrast_output = compute_contrast(labels, estimates, grp_contrast, contrast_type='t')

    
    #for printing the covariates
    if covariates == '':
        covariates = 'nothing'
        
    print('Processing done for: '+' '.join([contrast,ses,group,task]))
    print(f'Controlled for {covariates}')
    
    return (contrast_output, part_count)



In [ ]:
def get_excluded_runs(desired_outliers,contrast,ses,group,task):
    
    #empty list and dict to save runs to be excluded
    excluded_runs=[]
    excluded_runs_dict={}
    
    if len(desired_outliers.keys())==0:
        return excluded_runs_dict
    
    #read in cutoffs based on Q1-1.5*IQR and Q3+1.5*IQR based on all tasks and groups (not just current) for this session
    iqr_cutoffs_df = pd.read_csv(f'../../../derivatives/mriqc_summaries/iqr_cutoffs_ses-{ses}_rec-unco.tsv', low_memory=False, sep='\t')
    
    #read in tsv file with mriqc summary for current task, ses, group combi
    mriqc_summary = pd.read_csv(f'../../../derivatives/mriqc_summaries/group-{group}_ses-{ses}_task-{task}_rec-unco.tsv', low_memory=False, sep='\t')
    
    
    #possible outlier types that use IQR-based cutoff: tsnr, snr, gsr_x, gsr_y
    #exclude runs based on these if in desired_outliers list
    for outlier_type in ['tsnr', 'snr', 'gsr_x', 'gsr_y']:
        if outlier_type in desired_outliers.keys():
            #get upper and lower cutoffs from iqr summaries df
            lower_bound = iqr_cutoffs_df[outlier_type].iloc[0]
            upper_bound = iqr_cutoffs_df[outlier_type].iloc[1]
            
            #find subs_runs combinations outside of cutoffs 
            above_subs_runs = mriqc_summary['subs_runs'][mriqc_summary[outlier_type]>=upper_bound].tolist()
            below_subs_runs = mriqc_summary['subs_runs'][mriqc_summary[outlier_type]<=lower_bound].tolist()
            subs_runs = above_subs_runs+below_subs_runs
            excluded_runs+=subs_runs
            
    if 'fd_mean' in desired_outliers.keys():
        above_fd_mean_runs = mriqc_summary['subs_runs'][mriqc_summary['fd_mean']>desired_outliers['fd_mean']].tolist()
        if 'fd_perc' in desired_outliers:
            below_fd_perc_runs = set(mriqc_summary['subs_runs'][mriqc_summary['fd_perc']<desired_outliers['fd_perc']].tolist())
            above_fd_mean_runs = set(above_fd_mean_runs)
            above_fd_mean_runs = list(above_fd_mean_runs.difference(below_fd_perc_runs))
        excluded_runs+=above_fd_mean_runs
        
    #remove duplicates
    excluded_runs=list(set(excluded_runs))
    
    #turn into dict
    for sub_run in excluded_runs:
        sub = sub_run.split('_')[0]
        run = sub_run.split('_')[1]
        if sub in excluded_runs_dict.keys():
            excluded_runs_dict[sub].append(run)
        else:
            excluded_runs_dict[sub]=[run]
    
    return excluded_runs_dict
    

In [ ]:
def rm_CUD_baseline(effect_size_maps, all_subs):
    
    #subs to be excluded (only MM)
    excluded_subs = ['MM014','MM188','MM197','MM217','MM228','MM239','MM241']
    
    #get only effect size maps that aren't those of any of the excluded subjects
    final_effect_size_maps = [path for path in effect_size_maps if path.split('/sub-')[1].split('/')[0] not in excluded_subs]

    #get only subjects that aren't those of any of the excluded subjects
    final_all_subs = [sub for sub in all_subs if sub not in excluded_subs]

    return final_effect_size_maps, final_all_subs
    

In [ ]:
def select_subgroup(subgroup, effect_size_maps, subs_csv, subs):
    non_img_data = pd.read_csv(f"../../../sourcedata/non_imaging_data/MMJ-Processed_data-2022_05_27-13_58-6858bbe.csv",low_memory=False)
    final_subs = []
    final_subs_csv = []
    final_effect_size_maps = []
    if subgroup == 'dipstick_THC':
        dipstick_THC_results_dict = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'One year'].groupby('IDS.CHR.Subject')['URN.LGC.THC_present'].agg("first").to_dict()
        for i in range(len(effect_size_maps)):
            if subs_csv[i] in dipstick_THC_results_dict.keys():
                if dipstick_THC_results_dict[subs_csv[i]]:
                    final_subs.append(subs[i])
                    final_subs_csv.append(subs_csv[i])
                    final_effect_size_maps.append(effect_size_maps[i])
    
                
    return (final_effect_size_maps, final_subs_csv, final_subs)


In [ ]:
def get_effect_size_maps(contrast,ses,group,task,excluded_subs_runs_dict):
        
    #get individual effect size maps
    #get all second-level contrasts that exist
    second_level_effect_size_maps = glob.glob(f'../../../derivatives/task_analysis_surface/second_level/sub-{group}*/ses-{ses}/task-{task}/sub-{group}*_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_effect_size_fx.dscalar.nii')
    second_level_subs = [path.split('/sub-')[1].split('/')[0] for path in second_level_effect_size_maps if path]
    second_level_dict = dict(zip(second_level_subs, second_level_effect_size_maps))

    #remove any second level contrasts who have at least one run that needs to be excluded 
    for sub in excluded_subs_runs_dict.keys():
        if sub in second_level_dict.keys():
            second_level_dict.pop(sub)
    
    second_level_subs = list(second_level_dict.keys())
    second_level_effect_size_maps = list(second_level_dict.values())
    
    print(f'second level data subs count: {len(second_level_subs)}')
    
    #get first-level contrasts for the remaining subjects
    all_first_level_effect_size_maps = glob.glob(f'../../../derivatives/task_analysis_surface/first_level/sub-{group}*/ses-{ses}/task-{task}/sub-{group}*_ses-{ses}_task-{task}_rec-unco_run-*_contrast-{contrast}_effect_size.dscalar.nii')
    first_level_effect_size_maps = [path for path in all_first_level_effect_size_maps if path.split('/sub-')[1].split('/')[0] not in second_level_subs]
    first_level_subs = [path.split('/sub-')[1].split('/')[0] for path in first_level_effect_size_maps if path]
    first_level_runs = [path.split('run-')[1].split('_')[0] for path in first_level_effect_size_maps if path]

    #need to work with sub_run keys since otherwise we'd have duplicate keys
    first_level_subs_runs = ['_'.join(x) for x in zip(first_level_subs,first_level_runs)]
    first_level_dict = dict(zip(first_level_subs_runs, first_level_effect_size_maps))

    for sub,run_list in excluded_subs_runs_dict.items():
        for run in run_list:
            sub_run = f'{sub}_{run}'
            if sub_run in first_level_dict.keys():
                first_level_dict.pop(sub_run)
    
    first_level_subs = [sub_run.split('_')[0] for sub_run in first_level_dict.keys()]
    first_level_effect_size_maps = list(first_level_dict.values())
    
    print(f'first level data subs count: {len(first_level_subs)}')
    
    #combine the effect_size_maps into one list and the subjects into one list
    effect_size_maps = second_level_effect_size_maps + first_level_effect_size_maps
    
    all_subs = second_level_subs + first_level_subs
    
    print(f'all subs count: {len(all_subs)}')
    
    return (effect_size_maps, all_subs)


In [ ]:
def create_group_design_matrix(subs,group,ses,covariates):
    
    df_subs = pd.DataFrame(subs,columns=['subs'])
    non_img_data = pd.read_csv(f"../../../sourcedata/non_imaging_data/MMJ-Processed_data-2022_05_27-13_58-6858bbe.csv",low_memory=False)
    dummy_dfs = []
    
    if 'sex' not in covariates:
        #create design matrix with 1 column of 1s and as many rows as there are subjects
        group_design_matrix = pd.DataFrame([1] * len(subs), columns=['group_average'],)
    
    else:
        #create empty design matrix to add to
        group_design_matrix = pd.DataFrame()
        #add columns for male and female, that will then be combined to create the group average 
        grouped_sex = non_img_data.groupby("IDS.CHR.Subject")["SBJ.CHR.Sex"].agg("first")
        dict_sex = grouped_sex.to_dict()
        df_subs['sex'] = df_subs['subs'].map(dict_sex)
        dummy_dfs.append(pd.get_dummies(df_subs['sex']))
    
    #numericals can be added directly
    #mean center numerical values like age and CUDIT!

    if 'age' in covariates:
        #numerical can be added directly
        #mean center numerical value
        grouped_age = non_img_data.groupby("IDS.CHR.Subject")["SBJ.INT.Age"].agg("first")
        dict_age = grouped_age.to_dict()
        group_design_matrix['age'] = df_subs['subs'].map(dict_age)
        group_design_matrix['age'] = group_design_matrix['age'] - group_design_matrix['age'].mean()
    
    if 'total_cudit' in covariates:
        if group == 'HC': 
            grouped_HC_baseline_cudit = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Screening'].groupby('IDS.CHR.Subject')['INV.INT.CUDIT.Summed_score'].agg("first")
            dict_HC_baseline_cudit = grouped_HC_baseline_cudit.to_dict()
            group_design_matrix['total_cudit'] = df_subs['subs'].map(dict_HC_baseline_cudit)
            group_design_matrix['total_cudit'] = group_design_matrix['total_cudit'] - group_design_matrix['total_cudit'].mean()

        else:
            if ses == 'baseline':
                dict_MM_baseline_cudit = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Baseline'].groupby('IDS.CHR.Subject')['INV.INT.CUDIT.Summed_score'].agg("first").to_dict()
                group_design_matrix['total_cudit'] = df_subs['subs'].map(dict_MM_baseline_cudit)
                group_design_matrix['total_cudit'] = group_design_matrix['total_cudit'] - group_design_matrix['total_cudit'].mean()
                #needed because MM_141 is nan, so replacing with 0, which is the mean
                group_design_matrix['total_cudit'].fillna(0, inplace=True)
            else:
                dict_MM_1year_cudit = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'One year'].groupby('IDS.CHR.Subject')['INV.INT.CUDIT.Summed_score'].agg("first").to_dict()
                group_design_matrix['total_cudit'] = df_subs['subs'].map(dict_MM_1year_cudit)
                group_design_matrix['total_cudit'] = group_design_matrix['total_cudit'] - group_design_matrix['total_cudit'].mean()
    
    #encode the correspondence 
    freq_dict = {'Once or more per day':7,
        '5-6 days a week':6,
        '3-4 days a week':5,
        '1-2 days a week':4,
        'Less than once a week':3,
        'Less than once every two weeks':2,
        'Less than once a month':1,
        None:0}
    
    if 'THC_freq_month' in covariates:
        if group == 'HC':
            #results from screening visit (using this for consistency since CUDIT-R was also collected at screening visit)
            dict_HC_screening_THC = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Screening'].groupby('IDS.CHR.Subject')['TLF.CHR.THC.Frequency_in_month'].agg("last").to_dict()
            dict_HC_screening_THC_num = {k:freq_dict[v] for k,v in dict_HC_screening_THC.items()}
            group_design_matrix['THC_freq_month'] = df_subs['subs'].map(dict_HC_screening_THC_num)
            group_design_matrix['THC_freq_month'] = group_design_matrix['THC_freq_month'] - group_design_matrix['THC_freq_month'].mean()
        else:
            if ses == 'baseline':     
                #results from MRI visit (using this for consistency since CUDIT-R was also collected at MRI visit)
                dict_MM_MRIvisit_THC = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'Baseline'].groupby('IDS.CHR.Subject')['TLF.CHR.THC.Frequency_in_month'].agg("first").to_dict()
                dict_MM_MRIvisit_THC_num = {k:freq_dict[v] for k,v in dict_MM_MRIvisit_THC.items()}
                group_design_matrix['THC_freq_month'] = df_subs['subs'].map(dict_MM_MRIvisit_THC_num)
                group_design_matrix['THC_freq_month'] = group_design_matrix['THC_freq_month'] - group_design_matrix['THC_freq_month'].mean()
            else:
                dict_MM_MRIvisit_THC = non_img_data[non_img_data['SSS.CHR.Time_point'] == 'One year'].groupby('IDS.CHR.Subject')['TLF.CHR.THC.Frequency_in_month'].agg("first").to_dict()
                dict_MM_MRIvisit_THC_num = {k:freq_dict[v] for k,v in dict_MM_MRIvisit_THC.items()}
                group_design_matrix['THC_freq_month'] = df_subs['subs'].map(dict_MM_MRIvisit_THC_num)
                group_design_matrix['THC_freq_month'] = group_design_matrix['THC_freq_month'] - group_design_matrix['THC_freq_month'].mean()

                
    #add all items to design matrix that are still missing
    group_design_matrix = pd.concat([group_design_matrix]+dummy_dfs,axis=1,copy=False)
    
    return group_design_matrix



In [ ]:
def fdr_threshold_map(z_map):
    #alpha=.05, height_control='fdr'
    threshold = fdr_threshold(z_map, alpha=.05)
    #two-sided cutoff
    z_map[np.abs(z_map) < threshold] = 0
    return (z_map, threshold)


In [ ]:
#from neurohackademy cifti tutorial
def volume_from_cifti(data, axis):
    assert isinstance(axis, nibabel.cifti2.BrainModelAxis)
    data = data.T[axis.volume_mask]                          # Assume brainmodels axis is last, move it to front
    volmask = axis.volume_mask                               # Which indices on this axis are for voxels?
    vox_indices = tuple(axis.voxel[axis.volume_mask].T)      # ([x0, x1, ...], [y0, ...], [z0, ...])
    vol_data = np.zeros(axis.volume_shape + data.shape[1:],  # Volume + any extra dimensions
                        dtype=data.dtype)
    vol_data[vox_indices] = data                             # "Fancy indexing"
    return nibabel.Nifti1Image(vol_data, axis.affine)             # Add affine for spatial interpretation


In [ ]:
def plot_coronal_slices(threshold_map, threshold, MNI_template_used, contrast, group, ses, part_count):
    
    #get any one since they're all the same
    brain_model_axis = nibabel.load(glob.glob(f'../../../derivatives/task_analysis_surface/first_level/sub-{group}*/ses-{ses}/task-{task}/sub-{group}*_ses-{ses}_task-{task}_rec-unco_run-*_contrast-*_effect_size.dscalar.nii')[0]).header.get_axis(1)

    #make volume from subcortical part of cifti
    volume_threshold_map = volume_from_cifti(threshold_map, brain_model_axis)
    
    #create view with 4 coronal slices next to each other to visualiza basal ganglia
    display = plotting.plot_stat_map(volume_threshold_map, bg_img = MNI_template_used, display_mode='y', 
                                     cut_coords=[-10,-3,3,10], threshold=threshold, vmax=7, cmap='bwr')
    
    if threshold:
        rounded_thresh = round(threshold,2)
        display.title(text=f'Thresholded z map; "{contrast}" contrast of {group} at {ses}, exp. fdr = .05, z_thresh = {rounded_thresh}, n = {part_count}', size=14, y=1.2)
    else:
        display.title(text=f'Effect size map; "{contrast}" contrast of {group} at {ses}, n = {part_count}', size=14, y=1.2)
    
    return display


In [ ]:
def plot_surface_views(threshold_map, threshold, contrast, group, ses, part_count):
        
    right_hemi = plot_surf_stat_map(
        hcp.mesh.inflated_right, right_cortex_data(threshold_map[hcp.struct.cortex_right], fill=0), hemi='right',
        threshold=threshold, bg_map=hcp.mesh.sulc_right, 
        vmax=8, cmap='bwr',colorbar=False,darkness=0.6)
    
    right_flat = plot_surf_stat_map(
        hcp.mesh.flat_right, right_cortex_data(threshold_map[hcp.struct.cortex_right], fill=0), hemi='right',
        threshold=threshold, bg_map=hcp.mesh.sulc_right, 
        vmax=8, cmap='bwr',colorbar=False,darkness=0.6)
    
    left_hemi = plot_surf_stat_map(
        hcp.mesh.inflated_left, left_cortex_data(threshold_map[hcp.struct.cortex_left], fill=0), hemi='left',
        threshold=threshold, bg_map=hcp.mesh.sulc_left, 
        vmax=8, cmap='bwr',colorbar=False,darkness=0.6)
    
    left_flat = plot_surf_stat_map(
        hcp.mesh.flat_left, left_cortex_data(threshold_map[hcp.struct.cortex_left], fill=0), hemi='left',
        threshold=threshold, bg_map=hcp.mesh.sulc_left, 
        vmax=8, cmap='bwr',colorbar=False,darkness=0.6)
    
    return [right_hemi,right_flat,left_hemi,left_flat]

In [ ]:
def save_figures(coronal_display, right_hemi, right_flat, left_hemi, left_flat, threshold, group, ses, task, contrast, part_count):
    
    rounded_thresh = round(threshold,2)
    
    #check that output does not exist
    output = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_thresh}_display-*.png')
    if output: #checks if list is not empty
        print(f'At least partial group level output exists for group {group}, session {ses} and task {task}. This must be deleted for before generating new output.')
        return 
    
    
    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_surface','visualization','raw_figures')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    
    right_hemi.savefig(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_thresh}_n-{part_count}_display-righthemi.png')
    
    right_flat.savefig(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_thresh}_n-{part_count}_display-rightflat.png')
    
    left_hemi.savefig(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_thresh}_n-{part_count}_display-lefthemi.png')

    left_flat.savefig(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_thresh}_n-{part_count}_display-leftflat.png')
 
    coronal_display.savefig(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_thresh}_n-{part_count}_display-coronal.png')

    return



In [ ]:
task='mid'
sessions = ['baseline','1year']
groups = ['HC','MM']

#MNI_template_used = '../templates/tpl-MNI152NLin6Asym_res-01_desc-brain_T1w.nii.gz' #without skull
MNI_template_used = '../templates/tpl-MNI152NLin6Asym_res-01_T1w.nii.gz' #with skull


if task == 'mid':
    contrasts=['HiWin+LoWin-HiNoWin-LoNoWin','HiLoss+LoLoss-AvoidHiLoss-AvoidLoLoss']
           
elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','SuccStop-UnsuccStop']

elif task == 'nback':
    contrasts=['twoback-zeroback']

    
#options are age, sex, total_cudit, THC_freq_month at the moment
#note that if total_cudit or THC_freq_month are selected, the zmap is for their slopes, not for the group average slope
covariates = []

#subgroup = 'dipstick_THC'
subgroup = ''

#set to true if MM participants with CUD at baseline should be excluded from the analysis
rm_CUD = True

#set this to the ones we want to include with values for fd_mean and fd_perc as keys
desired_outliers = {'tsnr':[],'snr':[],'gsr_x':[],'gsr_y':[],'fd_mean':0.2,'fd_perc':0.3}
#desired_outliers = {}


#loop through all sessions, groups, contrasts
for ses in sessions:
    for group in groups:        
        for contrast in contrasts:
            #get group average group level contrast as the output
            contrast_output, part_count = get_group_average_contrast(contrast,ses,group,task,covariates,subgroup,rm_CUD,desired_outliers)
            print(part_count)
            #account for HC not having 1year scans
            if contrast_output:
                #compute FDR-corrected z-score thresholdmap
                #note that this does not have a cluster threshold
                threshold_map, threshold = fdr_threshold_map(contrast_output.z_score())
                
                #for "positive" activation difference (nicer visualization)
                if contrast == 'SuccStop-UnsuccStop':
                    contrast = 'UnsuccStop-SuccStop'
                    threshold_map = threshold_map*(-1)

                #thresholded displays
                coronal_display = plot_coronal_slices(threshold_map, threshold, MNI_template_used, contrast, group, ses, part_count)
                right_hemi,right_flat,left_hemi,left_flat = plot_surface_views(threshold_map, threshold, contrast, group, ses, part_count)
                
#                 #unthresholded displays
#                 coronal_display = plot_coronal_slices(contrast_output.z_score(), 0, MNI_template_used, contrast, group, ses, part_count)
#                 right_hemi,right_flat,left_hemi,left_flat = plot_surface_views(contrast_output.z_score(), 0, contrast, group, ses, part_count)
                
                
                #toggle this depending on if pngs should get saved of the plots
                save_figures(coronal_display,right_hemi,right_flat,left_hemi,left_flat,threshold, group, ses, task, contrast,part_count)

#toggle this depending on if pngs should be shown below
#plt.close('all')

In [ ]:
#group difference model for control vs. MM at baseline 

In [ ]:
def get_baseline_group_difference_contrast(contrast,ses,groups,task,covariates,subgroup, rm_CUD,desired_outliers):
    
    #get individual effect size maps and a design matrix per group
    effect_size_maps=[] #list of both groups effect size maps
    group_sizes=[] #list of the two group sizes
    group_design_matrices=[] #get separate design matrices per group that will be combined
    for group in groups:
        excluded_subs_by_group_runs_dict = get_excluded_runs(desired_outliers,contrast,ses,group,task)
        effect_size_maps_by_group, all_subs = get_effect_size_maps(contrast,ses,group,task,excluded_subs_by_group_runs_dict)
        
        #remove participants with CUD at baseline (only applies to MM)
        if rm_CUD:
            effect_size_maps_by_group, all_subs = rm_CUD_baseline(effect_size_maps_by_group, all_subs)
        
        #note that subject formatting is changed slightly to match that of the non_imaging_data csv file
        #need these to make design matrix
        subs_csv = ['_'.join([s for s in re.split(r'(MM|HC)', sub) if s]) for sub in all_subs]
       
        if subgroup:
            effect_size_maps_by_group, subs_csv, all_subs = select_subgroup(subgroup, effect_size_maps_by_group, subs_csv, all_subs)

        effect_size_maps += effect_size_maps_by_group
        group_sizes.append(len(effect_size_maps_by_group))
        
        #create design matrix per group
        #note that the columns that need to stay separate when stacking the design matrices later get renamed
        indv_design_matrix = create_group_design_matrix(subs_csv,group,ses,covariates)
        if 'sex' in covariates:
            indv_design_matrix.rename({'Female': f'Female_{group}', 'Male': f'Male_{group}'}, axis='columns',inplace=True)
        else:
            indv_design_matrix.rename({'group_average': f'group_average_{group}'}, axis='columns',inplace=True)
        group_design_matrices.append(indv_design_matrix)
    
    
    #stack design matrices of the two groups to make one large design matrix
    #note again that group_average or Male&Female columns, which were renamed, will not get stacked
    #thus, these will partially contain NaNs, which will be replaced by 0s
    group_design_matrix = pd.concat(group_design_matrices).replace(np. nan,0) 

    print(group_design_matrix)
    
    #grab masks needed (both groups at baseline)
    masks = glob.glob(f'../../../derivatives/ses-{ses}/sub-*/ses-{ses}/func/sub-*task-{task}*rec-unco*run-*_*{space}*brain_mask.nii.gz')

    #make intersection of the masks
    #threshold=1 corresponds to keeping the intersection of all masks, whereas threshold=0 is the union of all masks
    mask = intersect_masks(masks, threshold=1, connected=True)
    
    
    #define group level model
    cache_loc = './nilearn_cache' #change if desired

    second_level_model = SecondLevelModel(mask_img=mask, target_affine=None, target_shape=None, 
                                      smoothing_fwhm=None, memory=cache_loc, memory_level=1, 
                                      verbose=0, n_jobs=-2, minimize_memory=False)
    #fit model
    second_level_model = second_level_model.fit(
        effect_size_maps,
        design_matrix=group_design_matrix,
    )

    #make list of possible contrasts: HC only, MM only, MM-HC
    contrast_matrix = np.eye(group_design_matrix.shape[1])
    contrasts = dict([(column, contrast_matrix[i])
                  for i, column in enumerate(group_design_matrix.columns)])
    
    print(contrasts)
    
    grp_contrasts = {}
    
    for cov in covariates:
        if cov != 'sex':
            grp_contrasts[cov] = contrasts[cov]
        
    if 'sex' in covariates:
        grp_contrasts['sex'] = 0.5*contrasts['Female_MM']+0.5*contrasts['Male_MM']-0.5*contrasts['Female_HC']-0.5*contrasts['Male_HC']
    else:
        grp_contrasts['group_average'] = contrasts['group_average_MM']-contrasts['group_average_HC']
        
    print(grp_contrasts)
    
    
    contrast_outputs = {}
    
    for key in grp_contrasts.keys():
        contrast_output = second_level_model.compute_contrast(second_level_contrast=grp_contrasts[key], 
                                                          second_level_stat_type=None, output_type='all')
        contrast_outputs[key] = contrast_output
        
        
    #for printing the covariates
    if covariates == '':
        covariates = 'nothing'
        
    print('Processing done of the baseline difference of MM and HC for: '+' '.join([contrast,task]))
    print(f'Controlled for {covariates}')

    
    return (contrast_outputs, mask,(group_sizes[0],group_sizes[1]))



In [ ]:
groups=['MM','HC']
ses='baseline'
task='mid'
space='MNI152NLin6Asym' #change if desired
#covariates_list = [[],['sex'],['age'],['THC_freq_month'],['total_cudit'],['age','sex'],['THC_freq_month','age','sex'],['total_cudit','age','sex'] ]
covariates_list = [['sex','age']]
MNI_template_used = nibabel.load('../templates/tpl-MNI152NLin6Asym_res-02_desc-brain_T1w.nii.gz')


if task == 'mid':
    contrasts=['HiWin+LoWin-HiNoWin-LoNoWin','HiLoss+LoLoss-AvoidHiLoss-AvoidLoLoss']

elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','SuccStop-UnsuccStop']

elif task == 'nback':
    contrasts=['twoback-zeroback']

    
subgroup = ''
#subgroup = 'dipstick_THC'

#set to true if MM participants with CUD at baseline should be excluded from the analysis
rm_CUD = True

#set this to the ones we want to include with values for fd_mean and fd_perc as keys
desired_outliers = {'tsnr':[],'snr':[],'gsr_x':[],'gsr_y':[],'fd_mean':0.3,'fd_perc':0.3}
#desired_outliers = {}


#TO: loop through all contrasts
for covariates in covariates_list:
    for contrast in contrasts:
        #compute group difference contrast
        contrast_outputs, mask, part_count = get_baseline_group_difference_contrast(contrast,ses,groups,task,covariates,subgroup,rm_CUD,desired_outliers)
        print(covariates)
        
        print(f'We will print the covariate-based contrasts in the following order: {contrast_outputs.keys()}:')

        for key in contrast_outputs.keys():
            
            contrast_output = contrast_outputs[key]
            
            #calculate fdr-thresholded map
            threshold_map, threshold = fdr_threshold_map(contrast_output['z_score'], mask)
            
            #make visualizations
            axial_slices = plot_axial_slices(threshold_map, threshold, MNI_template_used, contrast, 'MM minus HC', ses, part_count)
            glass_brain = plot_glass_brains(threshold_map, threshold, contrast, 'MM minus HC', ses, part_count)

            #effect_axial_slices = plot_axial_slices(contrast_output['effect_size'], None, MNI_template_used, contrast, group, ses, part_count)



In [ ]:
#PAIRED group difference model for MM at 1year vs. baseline

In [ ]:
def get_MM_paired_group_difference_contrast(contrast,sessions,group,task,covariates,subgroup,rm_CUD,desired_outliers):
        
    #get individual effect size maps by subjects
    effect_size_maps_by_ses = [] #list of two dictionaries, one per ses, both with subjects as keys and their effect size paths as values
    
    for ses in sessions:
        excluded_subs_runs_dict = get_excluded_runs(desired_outliers,contrast,ses,group,task)
        
        effect_size_maps, all_subs = get_effect_size_maps(contrast,ses,group,task,excluded_subs_runs_dict)
        #remove participants with CUD at baseline (only applies to MM)
        if rm_CUD:
            effect_size_maps, all_subs = rm_CUD_baseline(effect_size_maps, all_subs)
        effect_size_by_subj_dict = dict(zip(all_subs, effect_size_maps))
        effect_size_maps_by_ses.append(effect_size_by_subj_dict)

    #find subjects with both baseline and 1year scans
    subjs_with_both_ses = list(set(effect_size_maps_by_ses[0].keys()).intersection(set(effect_size_maps_by_ses[1].keys())))
    
    #subtract baseline from 1year scans
    #save as list of nii effect size maps
    effect_size_arrays = []
    for sub in subjs_with_both_ses:
        effect_size_map_0 = effect_size_maps_by_ses[0][sub]
        effect_size_arr_0 = nimg.load_img(effect_size_map_0).get_fdata(dtype='f4')
        effect_size_map_1 = effect_size_maps_by_ses[1][sub]
        effect_size_arr_1 = nimg.load_img(effect_size_map_1).get_fdata(dtype='f4')
        diff_arr = effect_size_arr_0 - effect_size_arr_1
        effect_size_arrays.append(diff_arr)

    #rewrite subject names so they match the csv file
    subs_csv = ['_'.join([s for s in re.split(r'(MM|HC)', sub) if s]) for sub in subjs_with_both_ses]
    
    if subgroup:
        effect_size_arrays, subs_csv, subjs_with_both_ses = select_subgroup(subgroup, effect_size_arrays, subs_csv, subjs_with_both_ses)
    
    part_count = len(subjs_with_both_ses)
    
    #get design matrix for the group
    #if sex present, will include Male/Female; otherwise will include a column of all 1s called group average
    group_design_matrix = create_group_design_matrix(subs_csv,group,ses,covariates)


    effect_size_arrays_combined = np.vstack(effect_size_arrays)


    #set up and run glm
    #note that we are not smoothing with nilearn since the data was smoothed using the HCP pipeline
    labels, estimates = run_glm(effect_size_arrays_combined, group_design_matrix.values, noise_model='ols', 
                                n_jobs=-2, verbose=0)
  

    #define contrast matrix to include single column of 1s 
    contrast_matrix = np.eye(group_design_matrix.shape[1])
    contrasts = dict([(column, contrast_matrix[i])
                  for i, column in enumerate(group_design_matrix.columns)])
    
    print(contrasts)
    

    if 'sex' in covariates:
        grp_contrast = 0.5*contrasts['Female']+0.5*contrasts['Male']
    else:
        grp_contrast = contrasts['group_average']
        
    print(grp_contrast)

    
    #compute the group average contrast
    #pick stat_type none because it automatically selects t vs. F test 
    contrast_output = compute_contrast(labels, estimates, grp_contrast, contrast_type='t')

    
    #for printing the covariates
    if covariates == '':
        covariates = 'nothing'
        
    print('Processing done for: '+' '.join([contrast,ses,group,task]))
    print(f'Controlled for {covariates}')

    return (contrast_output, part_count)

    

In [ ]:
sessions=['1year','baseline']
task='mid'
group='MM'
#covariates_list = [['age','sex','THC_freq_month','total_cudit']]
covariates = []
space='MNI152NLin6Asym' #change if desired
MNI_template_used = nibabel.load('../templates/tpl-MNI152NLin6Asym_res-01_desc-brain_T1w.nii.gz')


if task == 'mid':
    contrasts=['HiWin+LoWin-HiNoWin-LoNoWin','HiLoss+LoLoss-AvoidHiLoss-AvoidLoLoss']

elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','SuccStop-UnsuccStop']

elif task == 'nback':
    contrasts=['twoback-zeroback']
    
    
subgroup = ''
#subgroup = 'dipstick_THC'

#set to true if MM participants with CUD at baseline should be excluded from the analysis
rm_CUD = True

#set this to the ones we want to include with values for fd_mean and fd_perc as keys
desired_outliers = {'tsnr':[],'snr':[],'gsr_x':[],'gsr_y':[],'fd_mean':0.3,'fd_perc':0.3}
#desired_outliers = {}


for contrast in contrasts:
    contrast_output, part_count = get_MM_paired_group_difference_contrast(contrast,sessions,group,task,covariates,subgroup,rm_CUD,desired_outliers)

    if contrast_output:

        #calculate fdr-thresholded map
        threshold_map, threshold = fdr_threshold_map(contrast_output.z_score())

        #make visualizations

        coronal_display = plot_coronal_slices(threshold_map, threshold, MNI_template_used, contrast, group, 'baseline', part_count)

        right_hemi,right_flat,left_hemi,left_flat = plot_surface_views(threshold_map, threshold, contrast, group, 'baseline', part_count)
         
        print(threshold)